In [272]:
model_name="neuralmind/bert-large-portuguese-cased"

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [3]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 47.4MB/s]


In [4]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [5]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━

In [6]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [7]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [273]:


models_list_3=[
        model_name,
          ]



models_name_3=[
        model_name,

]

In [274]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["ptbr"]
# langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [275]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [276]:
train_data=all_csv


In [277]:
train_data=train_data.fillna(0)

In [278]:
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')

In [279]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,ptbr_train_track_a_00001,0,0,0,0,0,0,ptbr,moça eu fiz uma análise com o tinder uma vez e...
1,1,ptbr_train_track_a_00002,1,1,0,0,0,0,ptbr,eles sempre mostram a míseria em como tudo era...
2,2,ptbr_train_track_a_00003,0,0,0,1,1,0,ptbr,eu nunca quis tanto algo como quero você jamai...
3,3,ptbr_train_track_a_00004,1,0,0,0,0,0,ptbr,esperando aqui o stf vim me emparedar do eu ex...
4,4,ptbr_train_track_a_00005,1,0,0,0,0,0,ptbr,e no final o ps vai pagar o pato ja é o 4° tre...
...,...,...,...,...,...,...,...,...,...,...
2221,2221,ptbr_train_track_a_02222,1,0,0,0,0,0,ptbr,fala fala fala mas fez pior da uma segurada ga...
2222,2222,ptbr_train_track_a_02223,0,0,0,0,0,0,ptbr,lampião era um bandido ou um cidadão louvável
2223,2223,ptbr_train_track_a_02224,1,0,1,0,1,0,ptbr,já somos um evangelistão sinto que voltamos a ...
2224,2224,ptbr_train_track_a_02225,0,0,0,0,0,0,ptbr,sim eu ia citar isso no comentário anterior ma...


In [280]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness']


In [281]:
train_data['lang'].value_counts()

,count
lang,
ptbr,2226


In [282]:
sum(train_data['lang'].isna())

0

In [283]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [284]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [285]:
maxi=train_data['clean_message'].apply(len).max()
maxi

2576

In [286]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
472,ptbr_train_track_a_00473,0,0,0,0,0,0,ptbr,o grande diferencial de um colégio não é só o ...


In [287]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['o grande diferencial de um colégio não é só o ensino mas a qualidade dos alunos que é o que vai te passar uma noção de média melhor por exemplo em um colégio excelente a média vai ser muito mas muito mais alta entre os alunos ah sim agora eu entendi o que você quis dizer sim talvez esse realmente seja o melhor dos casos mas eu fui fazer o vestibulinho pra provas de colégios desse estilo muito cedo grande parte da matéria eu acabava sem saber muito sobre e tals então atrapalha o ensino é certamente ótimo no colégio onde fui e muitos dos alunos vão bem em vestibular facilmente mas as atividades olímpicas estiveram falhando ao menos na época em que estudei também tem que mesmo sendo um ótimo colégio e renomado por ser particular e ter somente uma unidade não tem a mesma segregação que colégios objetivo ou etapa possuem a única unidade tem que englobar todos os tipos de alunos que é diferente dos exemplos onde os alunos vão para diferentes unidades praticamente baseado em seu nível

In [288]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [289]:
# train valid split
# train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [290]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [291]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [292]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1508      718  2226
disgust      2151       75  2226
fear         2117      109  2226
joy          1645      581  2226
sadness      1904      322  2226
surprise     2073      153  2226


In [293]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [294]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([ 1.5501, 14.8400, 10.2110,  1.9157,  3.4565,  7.2745], device='cuda:0')

In [295]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [296]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [297]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [298]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [299]:
USE_LORA=False

In [300]:
!pip install -q sentence_transformers

In [301]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [302]:
del embedding_model
for _ in range(10):
  torch.cuda.empty_cache()

In [303]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

neuralmind/bert-large-portuguese-cased


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [304]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [305]:
# a=

In [306]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [307]:
device

'cuda'

In [308]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [309]:

# Custom Dataset for batching
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [310]:
from torch.utils.data import DataLoader

In [311]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [312]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)


Extracting embeddings: 100%|██████████| 70/70 [01:14<00:00,  1.07s/it]


In [313]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [314]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [315]:
import xgboost as xgb

In [316]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [317]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [318]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [319]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [320]:
!rm -rf svm*

In [321]:
from sklearn.svm import SVC

In [322]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [323]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [324]:
!zip svm_models_pkl.zip svm_model_*.pkl

  adding: svm_model_anger.pkl (deflated 75%)
  adding: svm_model_disgust.pkl (deflated 75%)
  adding: svm_model_fear.pkl (deflated 75%)
  adding: svm_model_joy.pkl (deflated 75%)
  adding: svm_model_sadness.pkl (deflated 75%)
  adding: svm_model_surprise.pkl (deflated 75%)


In [325]:
kind="dev"

In [326]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,ptbr_dev_track_a_00001,0,0,0,0,0,0,ptbr,eu n sou o com quem vc conversava só fui mostr...
1,1,ptbr_dev_track_a_00002,0,0,0,0,0,0,ptbr,sei nem qual é mais feio
2,2,ptbr_dev_track_a_00003,0,0,0,0,0,0,ptbr,essa aqui q vou fazer
3,3,ptbr_dev_track_a_00004,0,0,0,0,0,0,ptbr,eu entendi bem um pastor pede pra outro pastor...
4,4,ptbr_dev_track_a_00005,0,0,0,0,0,0,ptbr,tá muito linda pqp
...,...,...,...,...,...,...,...,...,...,...
195,195,ptbr_dev_track_a_00196,0,0,0,0,0,0,ptbr,sempre irão aparecer pessoas assim mas cabe a ...
196,196,ptbr_dev_track_a_00197,0,0,0,0,0,0,ptbr,corno pra fazer a boa
197,197,ptbr_dev_track_a_00198,0,0,0,0,0,0,ptbr,ai vi tua irmã ontem mas eu tava mt louca na h...
198,198,ptbr_dev_track_a_00199,0,0,0,0,0,0,ptbr,hoje o dia está lindo perfeito para levar um t...


In [327]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [328]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [329]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 7/7 [00:04<00:00,  1.48it/s]


In [330]:
y_dev_flat = np.array(y_dev)

In [331]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: disgust
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [332]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [333]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [334]:
!rm -rf track*

In [335]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'ptbr' saved to track_a/pred_ptbr.csv


In [336]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/pred_ptbr.csv (deflated 90%)
